## Pulling data from TIDC

Printing out each county's spending per capita, from 2023 to 2019.

(Counts go from 2023 -> 2019)

In [1]:
import bs4, requests, time, csv

/Users/jolie/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Pull in all the data for all the counties and put them into a dictionary

In [2]:
baseURL = "https://tidc.tamu.edu/public.net/Reports/DataSheet.aspx?cid="
data = []

for i in range(1,255):
    #pull in html from each of the 254 counties' unique pages given the county id
    url = baseURL + str(i)
    html = requests.get(url).text
    soup = bs4.BeautifulSoup(html, 'html.parser')
    countyName = soup.find("span",attrs={"id":"ctl00_MainContent_CallbackPanel_rpReport_RPHT"}).text.split(" County")[0]
    
    #select the rows we're interested in: population and felony, misd, and total expenditures
    pop = soup.find("tr",attrs={"id":"ctl00_MainContent_CallbackPanel_rpReport_gvReport_DXDataRow0"}) 
    felExp = soup.find("tr",attrs={"id":"ctl00_MainContent_CallbackPanel_rpReport_gvReport_DXDataRow5"})
    misdExp = soup.find("tr",attrs={"id":"ctl00_MainContent_CallbackPanel_rpReport_gvReport_DXDataRow10"})
    totalExp = soup.find("tr",attrs={"id":"ctl00_MainContent_CallbackPanel_rpReport_gvReport_DXDataRow16"})
    
    popYrs = pop.find_all('td')
    felExpYrs = felExp.find_all('td')
    misdExpYrs = misdExp.find_all('td')
    totalExpYrs = totalExp.find_all('td')

    #create a dictionary where our data will live for each county
    countyData = {'county': countyName}

    #go through each year and assign data
    for i in range(2,7):
        year = 2023 - (i-2)
        popField = "pop_" + str(year)
        felExpField = "felExp_" + str(year)
        misdExpField = "misdExp_" + str(year)
        totalExpField = "totalExp_" + str(year)

        countyData[str(popField)] = popYrs[i].text
        countyData[str(felExpField)] = felExpYrs[i].text
        countyData[str(misdExpField)] = misdExpYrs[i].text
        countyData[str(totalExpField)] = totalExpYrs[i].text

    data.append(countyData)

    #wait a second between pages to be nice
    time.sleep(1.3)



Get the column names and dump the array into a csv file


In [3]:
columnNames = data[0].keys()

with open("tidc_county_spending.csv", "w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=columnNames)

    writer.writeheader()
    writer.writerows(data)

Celebrate!!!